In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("assignment2.ipynb")

# Assignment 2: Pandas
## Cleaning and Exploring Data with Pandas

## Assignment Modalities -- IMPORTANT -- READ CAREFULLY

This assignment contains a number of question that are worth 1--2 points each, as seen at the table below. At each question, you need to fill in the missing code, which is denoted by `...`. In some cases, you could write a single line of code, in other cases you might want to use more lines to achieve the desired outcome. **NOTE** that you should not change any code segment except the part denoted as `...`, as it might interfere with the auto grading, see below.

The assignment will be automatically graded by running tests on the provided code. Some of the tests are available after each cell with missing code. To run the automatic grader, you need to have otter-grader installed. This is not mandatory, but advised, so that you can check the validity of your answers. **NOTE** that for the final grading we will run additional tests not available in this notebook.

The installation instructions for otter-grader are here:  
<https://otter-grader.readthedocs.io/en/latest/#installation>

**IMPORTANT**: First rename this notebook to `groupXX_assignment2.ipynb`, where `XX` should be your group number for the 2nd assignment, e.g., `01`. For the autograder to work locally, make sure that in the initialization of otter the right notebook name is passed:
```
grader = otter.Notebook("groupXX_assignment2.ipynb")
```

**Note**: Installing the otter-grader is not necessary. All tests visible to you are included in the notebook as assertions.

After finishing the assignment, you should submit this notebook named `groupXX_assignment2.ipynb`.


## Score Breakdown 
Question | Points
--- | ---
1a | 1
1b | 2
1c | 1
2a | 2
2b | 1
2ci | 1
2cii | 1
2d | 2
2e | 2
3a | 1
3bi | 2
3bii | 2
3ci | 1
3cii | 1
3ciii | 1
3civ | 1
4a | 2
4b | 3
4c | 2
Total | 29


## Assignment Objective

In this homework, we will investigate restaurant food safety scores for restaurants in San Francisco. The scores and violation information have been [made available by the San Francisco Department of Public Health](https://data.sfgov.org/Health-and-Social-Services/Restaurant-Scores-LIVES-Standard/pyih-qa8i). The main goal for this assignment is to walk through the process of Data Cleaning and EDA. 


As we clean and explore these data, you will gain practice with:
* Reading simple csv files and using Pandas
* Working with data at different levels of granularity
* Identifying the type of data collected, missing values, anomalies, etc.



## Before You Start

For each question in the assignment, please write down your answer in the answer cell(s) right below the question. 

We understand that it is helpful to have extra cells breaking down the process towards reaching your final answer. If you happen to create new cells below your answer to run codes, **NEVER** add cells between a question cell and the answer cell below it. It will cause errors when we run the autograder, and it will sometimes cause a failure to generate the PDF file.

**Important note: The local autograder tests will not be comprehensive. You can pass the automated tests in your notebook but still fail tests in the autograder.** Please be sure to check your results carefully.

Finally, unless we state otherwise, try to avoid using python for loops or list comprehensions.  The majority of this assignment can be done using builtin commands in Pandas and numpy.  


In [ ]:
import numpy as np
import pandas as pd

from pathlib import Path
import os # Used to interact with the file system

## Obtaining the Data

### File Systems and I/O



In general, we will focus on using python commands to investigate files.  However, it can sometimes be easier to use shell commands in your local operating system.  The following cells demonstrate how to do this.

**NOTE** Do not change the location of the data.

In [ ]:
from pathlib import Path

data_dir = Path('./data')



## Loading Food Safety Data

We have data, but we don't have any specific questions about the data yet. Let's focus on understanding the structure of the data; this involves answering questions such as:

* Is the data in a standard format or encoding?
* Is the data organized in records?
* What are the fields in each record?

Let's start by looking at the contents of the `data` dictionary.

## Looking At the Data Files

The following codeblocks are setup. Simply run the cells; **do not modify them**. Question 1a is where you will start to write code.


You should see five CSV files. Open up `legend.csv` to see its contents.  You should see something that looks like:

    "Minimum_Score","Maximum_Score","Description"
    0,70,"Poor"
    71,85,"Needs Improvement"
    86,90,"Adequate"
    91,100,"Good"

## Reading in and Verifying Data

Based on the above information, let's attempt to load `bus.csv`, `ins2vio.csv`, `ins.csv`, and `vio.csv` into pandas dataframes with the following names: `bus`, `ins2vio`, `ins`, and `vio` respectively.

*Note:* Because of character encoding issues one of the files (`bus`) will require an additional argument `encoding='ISO-8859-1'` when calling `pd.read_csv`. At some point in your future, you should read all about [character encodings](https://diveintopython3.problemsolving.io/strings.html).

In [ ]:
# path to directory containing data

bus = pd.read_csv(data_dir/'bus.csv', encoding='ISO-8859-1')
ins2vio = pd.read_csv(data_dir/'ins2vio.csv')
ins = pd.read_csv(data_dir/'ins.csv')
vio = pd.read_csv(data_dir/'vio.csv')

#This code is essential for the autograder to function properly. Do not edit
ins_test = ins

Now that you've read in the files, let's try some `pd.DataFrame` methods ([docs](https://pandas.pydata.org/pandas-docs/version/0.21/generated/pandas.DataFrame.html)).
Use the `DataFrame.head` method to show the top few lines of the `bus`, `ins`, and `vio` dataframes. To show multiple return outputs in one single cell, you can use `display()`. Currently, running the cell below will display the first few lines of the `bus` dataframe. 

In [ ]:
bus.head()

The `DataFrame.describe` method can also be handy for computing summaries of numeric columns of our dataframes. Try it out with each of our 4 dataframes. Below, we have used the method to give a summary of the `bus` dataframe. 

In [ ]:
bus.describe()

Now, we perform some sanity checks for you to verify that the data was loaded with the correct structure. Run the following cells to load some basic utilities (you do not need to change these at all):

First, we check the basic structure of the data frames you created:

In [ ]:
assert all(bus.columns == ['business id column', 'name', 'address', 'city', 'state', 'postal_code',
                           'latitude', 'longitude', 'phone_number'])
assert 6250 <= len(bus) <= 6260

assert all(ins.columns == ['iid', 'date', 'score', 'type'])
assert 26660 <= len(ins) <= 26670

assert all(vio.columns == ['description', 'risk_category', 'vid'])
assert 60 <= len(vio) <= 65

assert all(ins2vio.columns == ['iid', 'vid'])
assert 40210 <= len(ins2vio) <= 40220

Next we'll check that the statistics match what we expect. The following are hard-coded statistical summaries of the correct data.

In [ ]:
bus_summary = pd.DataFrame(**{'columns': ['business id column', 'latitude', 'longitude'],
 'data': {'business id column': {'50%': 75685.0, 'max': 102705.0, 'min': 19.0},
  'latitude': {'50%': -9999.0, 'max': 37.824494, 'min': -9999.0},
  'longitude': {'50%': -9999.0,
   'max': 0.0,
   'min': -9999.0}},
 'index': ['min', '50%', 'max']})

ins_summary = pd.DataFrame(**{'columns': ['score'],
 'data': {'score': {'50%': 76.0, 'max': 100.0, 'min': -1.0}},
 'index': ['min', '50%', 'max']})

vio_summary = pd.DataFrame(**{'columns': ['vid'],
 'data': {'vid': {'50%': 103135.0, 'max': 103177.0, 'min': 103102.0}},
 'index': ['min', '50%', 'max']})

from IPython.display import display

print('What we expect from your Businesses dataframe:')
display(bus_summary)
print('What we expect from your Inspections dataframe:')
display(ins_summary)
print('What we expect from your Violations dataframe:')
display(vio_summary)

The code below defines a testing function that we'll use to verify that your data has the same statistics as what we expect. Run these cells to define the function. The `df_allclose` function has this name because we are verifying that all of the statistics for your dataframe are close to the expected values. Why not `df_allequal`? It's a bad idea in almost all cases to compare two floating point values like 37.780435, as rounding error can cause spurious failures.

In [ ]:
"""Run this cell to load this utility comparison function that we will use in various
tests below (both tests you can see and those we run internally for grading).

Do not modify the function in any way.
"""


def df_allclose(actual, desired, columns=None, rtol=5e-2):
    """Compare selected columns of two dataframes on a few summary statistics.
    
    Compute the min, median and max of the two dataframes on the given columns, and compare
    that they match numerically to the given relative tolerance.
    
    If they don't match, an AssertionError is raised (by `numpy.testing`).
    """    
    # summary statistics to compare on
    stats = ['min', '50%', 'max']
    
    # For the desired values, we can provide a full DF with the same structure as
    # the actual data, or pre-computed summary statistics.
    # We assume a pre-computed summary was provided if columns is None. In that case, 
    # `desired` *must* have the same structure as the actual's summary
    if columns is None:
        des = desired
        columns = desired.columns
    else:
        des = desired[columns].describe().loc[stats]

    # Extract summary stats from actual DF
    act = actual[columns].describe().loc[stats]

    return np.allclose(act, des, rtol)

We will now explore each file in turn, including determining its granularity and primary keys and exploring many of the variables individually. Let's begin with the businesses file, which has been read into the `bus` dataframe.

<br/><br/><br/>

---

<br/><br/><br/>

## Question 1a: Examining the Business Data File

From its name alone, we expect the `bus.csv` file to contain information about the restaurants. Let's investigate the granularity of this dataset.

In [ ]:
bus.head()

The `bus` dataframe contains a column called `business id column` which probably corresponds to a unique business id.  However, we will first rename that column to `bid` for simplicity.

**Note**: In practice we might want to do this renaming when the table is loaded but for grading purposes we will do it here.


In [ ]:
bus = bus.rename(columns={"business id column": "bid"})

Examining the entries in `bus`, is the `bid` unique for each record (i.e. each row of data)? Your code should compute the answer, i.e. don't just hard code `True` or `False`.

Hint: use `value_counts()` or `unique()` to determine if the `bid` series has any duplicates.

In [ ]:
is_bid_unique = ...

is_bid_unique

In [ ]:
assert(is_bid_unique)

In [ ]:
grader.check("q1a")

<br/><br/><br/>

---


## Question 1b

We will now work with some important fields in `bus`.

1. Assign `top_names` to a list containing the top 5 most frequently used business names, from most frequent to least frequent.
2. Assign `top_addresses` to a list containing the top 5 addressses where businesses are located, from most popular to least popular.

Hint: you may find `value_counts()` helpful.

In [ ]:
top_names = ...
top_addresses = ...

top_names, top_addresses

In [ ]:
assert len(top_names) == 5
assert top_names[0] == "Peet's Coffee & Tea"

In [ ]:
grader.check("q1b")

<br/><br/><br/>

---


## Question 1c

Based on the above exploration, what does each record represent?

A. "One location of a restaurant."
B. "A chain of restaurants."
C. "A city block."

Answer in the following cell. Your answer should be a string, either `"A"`, `"B"`, or `"C"`.

In [ ]:
# What does each record represent?  Valid answers are:
#    A "One location of a restaurant."
#    B "A chain of restaurants."
#    C "A city block."
q1c = ...

In [ ]:
assert(q1c in set(["A", "B", "C"]))

In [ ]:
grader.check("q1c")

<br/><br/><br/>

---

<br/><br/><br/>

# 2: Cleaning the Business Data Postal Codes

The business data contains postal code information that we can use to aggregate the ratings over regions of the city.  Let's examine and clean the postal code field.

*NOTE*: In USA, postal codes have 5 digits, like `90210`. There is also a more specific or full postal code that has four additional digits, like `90210-1007`.

The postal code (sometimes also called a ZIP code) partitions the city into regions:

<img src="https://www.usmapguide.com/wp-content/uploads/2019/03/printable-san-francisco-zip-code-map.jpg" alt="ZIP Code Map" style="width: 600px">

<br/><br/><br/>

---


## Question 2a

How many restaurants are in each ZIP code? 

In the cell below, create a **series** where the index is the postal code and the value is the number of records with that postal code in descending order of count. You may need to use `groupby()`, `size()`, or `value_counts()`. Do you notice any odd/invalid zip codes?

<!--
BEGIN QUESTION
name: q2a
points: 2
-->

In [ ]:
zip_counts = ...
print(zip_counts)

In [ ]:
assert(type(zip_counts) == pd.Series)
assert(zip_counts["94103"] == 562)

In [ ]:
grader.check("q2a")

<br/><br/><br/>

--- 

## Question 2b

Answer the following question about the `postal_code` column in the `bus` dataframe.

   
1. What Python data type is used to represent a ZIP code?  
    A. `str`  
    B. `int`  
    C. `bool`  
    D. `float`

*Note*: ZIP codes and postal codes are the same thing.

Please write your answers in the cell below. Your answer should be a string, either `"A"`, `"B"`, `"C"`, or `"D"`.

In [ ]:
# What Python data type is used to represent a ZIP code? 
q2b = ...

In [ ]:
assert(q2b in set(["A", "B", "C"]))

In [ ]:
grader.check("q2b")

<br/><br/><br/>

--- 

## Question 2c

In question 2a we noticed a large number of potentially invalid ZIP codes (e.g., "Ca").  These are likely due to data entry errors.  To get a better understanding of the potential errors in the zip codes we will:

1. Import a list of valid San Francisco ZIP codes by using `pd.read_json` to load the file `data/sf_zipcodes.json` and extract a **series** of type `str` containing the valid ZIP codes.  *Hint: set `dtype` when invoking `read_json`.*
1. Construct a `DataFrame` containing only the businesses which DO NOT have valid ZIP codes.  You will probably want to use the `Series.isin` function. 


**Step 1**

<!--
BEGIN QUESTION
name: q2ci
points: 1
-->

In [ ]:
valid_zips = ...

valid_zips.head()

In [ ]:
assert( (valid_zips.dtype == object) | (valid_zips.dtype == pd.StringDtype())) 
assert(type(valid_zips) == pd.Series)

In [ ]:
grader.check("q2ci")

**Step 2**

<!--
BEGIN QUESTION
name: q2cii
points: 1
-->

In [ ]:
# has_valid_zip should be a boolean array
# A True value would indicate the business has a valid ZIP code

has_valid_zip = ...

invalid_zip_bus = ...
invalid_zip_bus.head(20)

In [ ]:
assert(type(invalid_zip_bus) == pd.DataFrame) 

In [ ]:
grader.check("q2cii")

<br/><br/><br/>

--- 

## Question 2d

In the previous question, many of the businesses had a common invalid postal code that was likely used to encode a MISSING postal code.  Do they all share a potentially "interesting address"?

In the following cell, construct a **series** that counts the number of businesses at each `address` that have this single likely MISSING postal code value.  Order the series in descending order by count. 

After examining the output, please answer the following question (2e) by filling in the appropriate variable. If we were to drop businesses with MISSING postal code values would a particular class of business be affected? 

In [ ]:
missing_zip_address_count = ...
missing_zip_address_count

In [ ]:
assert(type(missing_zip_address_count) == pd.Series)
assert(missing_zip_address_count['3914 Judah St'] == 1)

In [ ]:
grader.check("q2d")

<!-- END QUESTION -->

<br/><br/><br/>

--- 

## Question 2e

Examine the `invalid_zip_bus` dataframe we computed above and look at the businesses that DO NOT have the special MISSING ZIP code value. Some of the invalid postal codes are just the full 9 digit code rather than the first 5 digits. Create a new column named `postal5` in the original `bus` dataframe which contains only the first 5 digits of the `postal_code` column.

Then, for any of the `postal5` ZIP code entries that were not a valid San Fransisco ZIP Code (according to `valid_zips`), the provided code will set the `postal5` value to `None`.  

**Do not modify the provided code!**

In [ ]:
bus['postal5'] = ...


bus.loc[~bus['postal5'].isin(valid_zips), 'postal5'] = None
# Checking the corrected postal5 column
bus.loc[invalid_zip_bus.index, ['bid', 'name', 'postal_code', 'postal5']]

In [ ]:
assert('postal5' in bus.columns) 
assert((bus['postal5'].str.len() != 5).sum() == 221)

In [ ]:
grader.check("q2e")

<br/><br/><br/>

---

<br/><br/><br/>

# 3: Investigate the Inspection Data

Let's now turn to the inspection DataFrame. Earlier, we found that `ins` has 4 columns named 
`iid`, `score`, `date` and `type`.  In this section, we determine the granularity of `ins` and investigate the kinds of information provided for the inspections. 

Let's start by looking again at the first 5 rows of `ins` to see what we're working with.

In [ ]:
ins.head(5)

<br/><br/><br/>

---

## Question 3a

The column `iid` probably corresponds to an inspection id.  Is it a primary key?  Write an expression (line of code) that evaluates to `True` or `False` based on whether all the values are unique.

**Hint:** This is a very similar question to Question 1a.

In [ ]:
is_ins_iid_a_primary_key = ...
print(is_ins_iid_a_primary_key)

In [ ]:
assert(type(is_ins_iid_a_primary_key) == bool or type(is_ins_iid_a_primary_key) == np.bool) 

In [ ]:
grader.check("q3a")

<br/><br/><br/>

---

## Question 3b

The column `iid` appears to be the composition of two numbers and the first number looks like a business id.  

**Part 1.**: Create a new column called `bid` in the `ins` dataframe containing just the business id.  You will want to use `ins['iid'].str` operations to do this.  Also be sure to convert the type of this column to `int`

**Part 2.**: Then compute how many values in this new column are invalid business ids (i.e. do not appear in the `bus['bid']` column). This is verifying a foreign key relationship. Consider using the `pd.Series.isin` function.


**No python `for` loops or list comprehensions required!**

**Part 1**

In [ ]:
...
ins.head()

In [ ]:
assert('bid' in ins.columns) 
assert(ins['bid'].dtype == int)

In [ ]:
grader.check("q3bi")

**Part 2**

In [ ]:
invalid_bid_count = ...

print(invalid_bid_count)

In [ ]:
assert(invalid_bid_count == 0) 

In [ ]:
grader.check("q3bii")

<br/><br/><br/>

---

## Question 3c

What if we are interested in a time component of the inspection data?  We need to examine the date column of each inspection. 

**Part 1:** What is the type of the individual `ins['date']` entries? You may want to grab the very first entry and use the `type` function in python. 

**Part 2:** Use `pd.to_datetime` to create a new `ins['timestamp']` column containing of `pd.Timestamp` objects.  These will allow us to do more date manipulation.

**Part 3:** What are the earliest and latest dates in our inspection data?  *Hint: you can use `min` and `max` on dates of the correct type.*

**Part 4:** We probably want to examine the inspections by year. Create an additional `ins['year']` column containing just the year of the inspection.  Consider using `pd.Series.dt.year` to do this.  

**Note: If you get a `SettingWithCopyWarning`, consider using the [`assign` method](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.assign.html) to add a column to a DataFrame.**

**No python `for` loops or list comprehensions required!**

**Part 1**

In [ ]:
ins_date_type = ...
ins_date_type

In [ ]:
assert(type(ins_date_type) == type) 
assert(ins_date_type in [type("str"), type("int")])

In [ ]:
grader.check("q3ci")

**Part 2**

In [ ]:
...
ins.head()

In [ ]:
assert(type(ins['timestamp'][1]) == pd.Timestamp) 

In [ ]:
grader.check("q3cii")

**Part 3**

In [ ]:
earliest_date = ...
latest_date = ...

print("Earliest Date:", earliest_date)
print("Latest Date:", latest_date)

In [ ]:
assert(type(earliest_date) == pd.Timestamp) 
assert(type(latest_date) == pd.Timestamp)

In [ ]:
grader.check("q3ciii")

**Part 4**

In [ ]:
...

ins.head()

In [ ]:
assert('year' in ins.columns) 

In [ ]:
grader.check("q3civ")

<br/><br/><br/>

---

<br/><br/><br/>

# 4: Joining Data Across Tables

In this question we will start to connect data across mulitple tables.  We will be using the `merge` function. 

<br/><br/><br/>

--- 

## Question 4a

Let's figure out which restaurants had the lowest scores. Before we proceed, let's filter out missing scores from `ins` so that negative scores don't influence our results.

In [ ]:
ins = ins[ins["score"] > 0]

We'll start by creating a new dataframe called `ins_named`. It should be exactly the same as `ins`, except that it should have the name and address of every business, as determined by the `bus` dataframe. If a `business_id` in `ins` does not exist in `bus`, the name and address should be given as `NaN`. 

*Hint*: Use the merge method to join the `ins` dataframe with the appropriate portion of the `bus` dataframe. See the official [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) on how to use `merge`.

*Note*: For quick reference, a pandas 'left' join keeps the keys from the left frame, so if `ins` is the left frame, all the keys from `ins` are kept and if a set of these keys don't have matches in the other frame, the columns from the other frame for these "unmatched" key rows contains NaNs.

In [ ]:
ins_named = ...


ins_named.head()

In [ ]:
assert("name" in ins_named and "address" in ins_named) 

In [ ]:
grader.check("q4a")

<br/><br/><br/>

--- 

## Question 4b

Let's look at the 20 businesses with the lowest **median** score.  Order your results by the median score followed by the business name to break ties. The resulting table should look like:


*Hint: You may find the `as_index` argument in the `groupby` method important*. [The documentation is linked here!](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.groupby.html)

<table border="1" class="dataframe">  <thead>    <tr style="text-align: right;">      <th>bid</th>      <th>name</th>      <th>median score</th>    </tr>  </thead>  <tbody>    <tr>      <th>84590</th>      <td>Chaat Corner</td>      <td>54.0</td>    </tr>    <tr>        <th>90622</th>      <td>Taqueria Lolita</td>      <td>57.0</td>    </tr>    <tr>         <th>94351</th>      <td>VBowls LLC</td>      <td>58.0</td>    </tr>    <tr>          <th>69282</th>      <td>New Jumbo Seafood Restaurant</td>      <td>60.5</td>    </tr>    <tr>         <th>1154</th>      <td>SUNFLOWER RESTAURANT</td>      <td>63.5</td>    </tr>  <tr>          <th>93150</th>      <td>Chez Beesen</td>      <td>64.0</td>    </tr>   <tr>     <th>39776</th>      <td>Duc Loi Supermarket</td>      <td>64.0</td>    </tr>  <tr>         <th>78328</th>      <td>Golden Wok</td>      <td>64.0</td>    </tr>  <tr>          <th>69397</th>      <td>Minna SF Group LLC</td>      <td>64.0</td>    </tr>     <tr>        <th>93502</th>      <td>Smoky Man</td>      <td>64.0</td>    </tr>    <tr>           <th>98995</th>      <td>Vallarta's Taco Bar</td>      <td>64.0</td>    </tr>    <tr>         <th>10877</th>      <td>CHINA FIRST INC.</td>      <td>64.5</td>    </tr>    <tr>        <th>71310</th>      <td>Golden King Vietnamese Restaurant</td>      <td>64.5</td>    </tr>     <tr>          <th>89070</th>      <td>Lafayette Coffee Shop</td>      <td>64.5</td>    </tr>
    <tr>          <th>71008</th>      <td>House of Pancakes</td>      <td>65.0</td>    </tr> <tr>         <th>2542</th>      <td>PETER D'S RESTAURANT</td>      <td>65.0</td>    </tr>            <tr>        <th>3862</th>      <td>IMPERIAL GARDEN SEAFOOD RESTAURANT</td>      <td>66.0</td>    </tr>    <tr>         <th>61427</th>      <td>Nick's Foods</td>      <td>66.0</td>    </tr>    <tr>          <th>72176</th>      <td>Wolfes Lunch</td>      <td>66.0</td>    </tr>    <tr>        <th>89141</th>      <td>Cha Cha Cha on Mission</td>      <td>66.5</td>    </tr>  </tbody></table>


In [ ]:
twenty_lowest_scoring = ...


twenty_lowest_scoring

In [ ]:
assert(set(twenty_lowest_scoring.columns) == {'median score', 'name'}) 

In [ ]:
grader.check("q4b")

### Question 4c

Let's figure out which restaurant had the worst score ever (single lowest score). 

In the cell below, assign `worst_restaurant` to the name of the restaurant with the **lowest inspection score ever**. Feel free to head to yelp.com and look up the reviews page for this restaurant.

In [ ]:
worst_restaurant = ...
worst_restaurant

In [ ]:
assert(type(worst_restaurant) == str) 
assert(len(worst_restaurant) > 0)

In [ ]:
grader.check("q4c")

## Congratulations! You have finished Assignment 2! ##